In [ ]:
#!jupyter nbconvert --to=python DIC_Glodap_Comparison_python3.ipynb

In [ ]:
class DICcomp:
    '''
    class DICcomp
    
    Compare DIC to Glodap
    
    -use layerwise = True to compare a set of depth
    -use depth_limit to specify maximum depth for mean-over-depth comparison
    '''
    def __init__(self,runname,resultpath,savepath,meshpath,ncpath,first_year,last_year,
                 GLODAPvar='TCO2',
                 mapproj='pc',
                 cmap = 'viridis',
                 savefig=False,
                 layerwise=False,depth_array=[],
                 depth_limit = 100):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.meshpath = meshpath
        self.ncpath = ncpath
        self.fyear = first_year
        self.lyear = last_year
        self.GLODAPvar = GLODAPvar
        self.mapproj = mapproj
        self.cmap = cmap
        self.savefig = savefig
        self.layerwise = layerwise
        self.depth_array = depth_array
        self.depth_limit = depth_limit

        import matplotlib.pyplot as plt
        import numpy as np
        from netCDF4 import Dataset
        from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        #import pickle

        import pyfesom2 as pf

        from plot_Taylor_normalized import plt_Taylor_norm
        
        print('Processing {0}'.format(self.resultpath))

        # load FESOM mesh -------------------------------------------------------------------------------------
        mesh       = pf.load_mesh(meshpath)
        years = np.arange(self.fyear, self.lyear+1,1)

        # check variables
        #NCfesom = self.resultpath + '/DFe.'+self.runname+'.'+str(self.fyear)+'.nc'
        #!ncdump -h $NCfesom

        labelfesom = 'FESOM Dissolved Inorganic Carbon {0}-{1}'.format(self.fyear,self.lyear)
        unitfesom = 'DIC [mmol C m$^{-3}$]' # equals to mumol/L

        # load FESOM data -------------------------------------------------------------------------------------
        DICfesom = pf.get_data(resultpath, "DIC", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent = True)
        
        # load FESOM mesh diag -------------------------------------------------------------------------------
        meshdiag= self.resultpath+'/'+self.runname+'.mesh.diag.nc'
        #!ncdump -h $meshdiag

        diag = pf.get_meshdiag(mesh,meshdiag=meshdiag, runid=self.runname)
        #print(diag)
        #print(diag['Z']) # depth of layers
        mesh_depths = diag['Z'].values

        # load raw Glodap data -------------------------------------------------------------------------------------
        f          = Dataset(self.ncpath, 'r')
        DepthRaw   = -f.variables['Depth'][:]                               # Depth is negative
        TCO2Raw    = f.variables[self.GLODAPvar][:,:,:] # #'TCO2'
        TCO2Raw    = np.ma.filled(TCO2Raw, np.nan)                          # From masked array to numpy array
    
        TCO2Glo     = np.zeros(shape=(np.shape(TCO2Raw)))                   # Change longitude from 0:360 to -180:180 
        for i in range(0,len(DepthRaw)):
          TCO2Glo[i,:,:] = np.hstack((TCO2Raw[i,:,161:360],TCO2Raw[i,:,0:161]))
        
        x360       = np.arange(-179.5,180.,1.)
        y180       = np.arange(-89.5,89.6,1.)
        X360, Y180 = np.meshgrid(x360, y180)
        

        unitglodap = 'DIC [mmol C m$^{-3}$]'
        labelglodap = 'Glodap: Interpolated present-day DIC for initialization'
        
        # interpolate Glodap data -------------------------------------------------------------------------------------

        # check maximum depth in WOA compared to FESOM
        dmin_woa = np.min(DepthRaw)
        dmin_fesom = np.min(mesh.zlev)

        if(dmin_woa <= dmin_fesom):
            print('***\nDepth greater in Glodap ({0}) than in FESOM ({1})'.format(dmin_woa, dmin_fesom))
            ilev = len(mesh.zlev)
            max_zlev = mesh.zlev[ilev-1]
        else:
            print('***\nDepth greater in FESOM ({1}) than in Glodap ({0})'.format(dmin_woa, dmin_fesom))
            ilev = np.where(mesh.zlev >= dmin_woa)
            ilev = ilev[0][-1]
            max_zlev = mesh.zlev[ilev]

        print('Please consider choosing max depth level {0} with max depth at {1}!\n***'.format(ilev,max_zlev))
    

        # storage container
        glodap_int = np.zeros((len(mesh.zlev)-1,len(mesh.x2)))
        #print(np.shape(din_int))

        for k in range(0,len(mesh.zlev)-1): # -1 because midlevel is used...
            lev = mesh.zlev[k] # current FESOM depth
            try:
                nextlev = mesh.zlev[k+1] # next level needed because FESOM data is defined for midlevel??
            except:
                print('reached bottom... no midlevel could be calculated')
            finally:
                midlev = (lev + nextlev )/2 # FESOM data is expected to be at midlevel?!
                #print(lev)
            ind1 = np.where(DepthRaw >= midlev)
            ind1 = ind1[0][-1]
            ind2 = np.where(DepthRaw < midlev)[0]

            if ind2.size > 0:                            # If we have not yet reached the bottom
                ind2 = ind2[0]                           # The index of the depth level below the current fesom level
                c    = DepthRaw[ind1]-DepthRaw[ind2]     # Difference in depth between the data value above and below the fesom depth
                c1   = DepthRaw[ind1]-midlev                # Difference between fesom depth and data depth above
                c2   = -(DepthRaw[ind2]-midlev)             # Difference between fesom depth and data depth below
                c1   = (c-c1)/c                          # Scaling coefficient for the depth above
                c2   = (c-c2)/c                          # Scaling coefficient for the depth below
            else:                                        # We have reached the bottom
                c1   = 1.
                c2   = 0.
                ind2 = ind1

            indZ  = np.where(mesh.zlev == lev)                               
            # original code:
            # indZ  = np.where(-mesh.z3 == lev)          # Find the mesh index of the current fesom depth
            indZ = indZ[0] 
            if(False):
                print('\nFESOM depth = {0} -> midlevel = {8}, WOA depths = {1}, {2} \nDepth indices: {3} {4},  FESOM index: {5} \nScaling c1 = {6}, c2 = {7}'.format(lev,DepthRaw[ind1],DepthRaw[ind2],ind1, ind2,indZ,c1,c2,midlev))


            aux1  = TCO2Glo[ind1,:,:]                     # Find the data above the current fesom depth
            aux2  = TCO2Glo[ind2,:,:]                     # Find the data below the current fesom depth
            aux   = np.squeeze(c1*aux1+c2*aux2)          # Scaling the data according to vertical distribution as found above
            ind   = np.squeeze(~np.isnan(aux)) 
            #print(np.shape(aux), np.shape(ind))

            # first interpolation to original grid to close empty parts
            aux = griddata((X360[ind], Y180[ind]), aux[ind], (X360, Y180), method='nearest')                             
            # 2D field without nans                           

            # second interpolation to FESOM grid
            glodap_int[indZ,:] = griddata((X360.ravel(), Y180.ravel()), aux.ravel(), 
                                   (mesh.x2, mesh.y2), method='nearest')  
            # Final interpolated field

            if np.isnan(np.min(glodap_int)): print('WARNING: The interpolated field contains NaNs at depth',lev)                 # Testing if results contain NaNs. If yes, the routine needs adjustments

            if(False):
                print('Depth: {0} min = {1} max = {2} mean = {3}'.format(lev,np.min(glodap_int), np.max(glodap_int), np.mean(glodap_int)))

        glodap_int = np.swapaxes(glodap_int,0,1) # adjust axes layout to FESOM output
        

        # apply sea mask to Glodap as in FESOM ----------------------------------------------------------------------------------
        # assumption: there is no ocean where value in FESOM == 0
        glodap_int_ma = np.copy(glodap_int)
        glodap_int_ma[DICfesom == 0] = 0

        # plot Glodap and FESOM ----------------------------------------------------------------------------------
        if(self.layerwise):
            if(self.depth_array == []):
                depth_array = (0,50,200,1000,2000,4000)

            for d in depth_array:
                # get mesh index closest to desired depth
                i = pf.ind_for_depth(d,mesh) 
                # get midlevel depth
                plot_depth = str((mesh.zlev[i]+mesh.zlev[i+1])/2)

                if(True):
                    print('\nInput depth = {0}, plotting at level depth = {1} m\nFESOM min = {2}, max = {3}\nGlodap min = {4}, max = {5}'.format(
                        d,plot_depth,
                        np.nanmin(DICfesom[:,i]),np.nanmax(DICfesom[:,i]),
                        np.nanmin(glodap_int_ma[:,i]),np.nanmax(glodap_int_ma[:,i])))

                pf.plot(mesh, [glodap_int_ma[:,i],DICfesom[:,i], glodap_int_ma[:,i]-DICfesom[:,i]], 
                        rowscol= (1,3),
                        levels = (0,45,40),
                        units=unitglodap + '\n at depth = {0} m'.format(plot_depth), 
                        mapproj=self.mapproj, # robinson projection takes more time!
                        cmap = self.cmap,
                        titles=[labelwoa, labelfesom, 'Glodap - FESOM'],figsize = (20,20)
                       )

            # statistics  -------------------------------------------------------------------------------------
            # preparation of datasets
            if np.isnan(np.min(glodap_int)): print('WARNING: The interpolated WOA field contains NaNs at depth')
            if np.isnan(np.min(DICfesom)): print('WARNING: The interpolated FESOM field contains NaNs at depth')


            for d in depth_array:
                # get mesh index closest to desired depth
                i = pf.ind_for_depth(d,mesh) 
                # get midlevel depth
                plot_depth = str((mesh.zlev[i]+mesh.zlev[i+1])/2)

                title = 'Taylor Diagram for DIC at {0} m'.format(plot_depth)
                plt_Taylor_norm(glodap_int[i,:],DICfesom[i,:],mask=True,title=title)

                if False: #old
                    # get statistics only from ocean gridpoints (same mask assumption as above)
                    ind_stat = np.where(DICfesom[i,:] != 0)

                    taylor_stats1 = sm.taylor_statistics(glodap_int[i,ind_stat],DICfesom[i,ind_stat])
                    sdev = np.array([taylor_stats1['sdev'][0], taylor_stats1['sdev'][1]])
                    crmsd = np.array([taylor_stats1['crmsd'][0], taylor_stats1['crmsd'][1]])
                    ccoef = np.array([taylor_stats1['ccoef'][0], taylor_stats1['ccoef'][1]])

                    label = ['Observation', 'Model']# at {0} m'.format(plot_depth)]

                    fig = plt.figure(figsize=(7,7), facecolor='w', edgecolor='k')
                    sm.taylor_diagram(sdev,crmsd,ccoef, styleOBS = '-', colOBS = 'r', markerobs = 'o',
                                          titleOBS = 'observation', markerLabel = label,
                                          markerLabelColor = 'c',
                                          markerColor = 'c', markerLegend = 'on',
                                          tickRMS = range(0,5,1), tickRMSangle = 135.0,
                                          colRMS = 'm', styleRMS = ':', widthRMS = 2.0,
                                          titleRMS = 'off', tickSTD = range(0,10,2 ),
                                          axismax = 10.0, 
                                          colSTD = 'b', styleSTD = '-.',
                                          widthSTD = 1.0, titleSTD = 'on',
                                          colCOR = 'k', styleCOR = '--', widthCOR = 1.0,
                                          titleCOR = 'on')
                    plt.title('Taylor Diagram for DIC at {0} m'.format(plot_depth), loc='right')
        
            if(self.savefig==True): print('\n***\n***Too many figures to export...\n***')
        
        # mean over depth  -------------------------------------------------------------------------------------
        else:
            # get level depth index closest to depth_limit
            i_depth_limit = pf.ind_for_depth(self.depth_limit, mesh)
            
            # corresponding level depth
            depth_limit_level = - mesh.zlev[i_depth_limit]
            
            # corresponding layer depth (where concentrations are defined):
            depth_limit_layer = - mesh_depths[i_depth_limit]
            
            # print overview of chosen depth
            print('DIC as mean over depth \nwith max layer depth = {0} \n(level depth = {1}m, mesh index {2})'.format(
                depth_limit_layer,depth_limit_level,i_depth_limit))
            
            # mean over depth            
            DICfesom_mean = np.mean(DICfesom[:,:i_depth_limit], axis = 1)
            glodap_int_ma_mean = np.mean(glodap_int_ma[:,:i_depth_limit], axis = 1)

            print('\nFESOM min = {0:4.4f}, max = {1:4.4f}\nGLODAP min = {2:4.4f}, max = {3:4.4f}'.format(
                    np.nanmin(DICfesom_mean),np.nanmax(DICfesom_mean),
                    np.nanmin(glodap_int_ma_mean),np.nanmax(glodap_int_ma_mean)))
    
            fig_data = pf.plot(mesh, [glodap_int_ma_mean,DICfesom_mean],#, glodap_int_ma_mean-DICfesom_mean], 
                    rowscol= (1,2),
                    #levels = (0,35,36),
                    units=unitglodap, 
                    mapproj=self.mapproj, # robinson projection takes more time!
                    cmap = self.cmap,
                    titles=[labelglodap, 
                            'Mean over depth (max={2}m)\nFESOM Dissolved Inorganic Carbon ({0}-{1})'.format(
                                first_year,last_year,depth_limit_layer),
                            ],
                    figsize = (20,20)
                   )
            # fig export  -------------------------------------------------------------------------------------
            if(self.savefig==True):
                plt.savefig(self.savepath+self.runname+'_'+'DIC_Glodap_'+str(years[0])+'to'+str(years[-1])+'.png', 
                        dpi = 300, bbox_inches='tight')
            plt.show(block=False)  
            
            fig_diff = pf.plot(mesh, [glodap_int_ma_mean-DICfesom_mean], 
                    levels = (-150,150,301),
                    units=unitglodap, 
                    mapproj=self.mapproj, # robinson projection takes more time!
                    cmap = 'RdBu',
                    titles='Initialization fields minus FESOM \n(Mean over depth, max={0}m)'.format(
                        depth_limit_layer),
                    figsize = (10,10)
                   )
                
            # statistics  -------------------------------------------------------------------------------------            
            # preparation of datasets
            if np.isnan(np.min(glodap_int_ma_mean)): print('WARNING: The interpolated Glodap field contains NaNs at depth')
            if np.isnan(np.min(DICfesom_mean)): print('WARNING: The interpolated FESOM field contains NaNs at depth')


            title = 'Taylor Diagram for DIC \n(mean over depth, max = {0}m)'.format(depth_limit_layer)
            plt_Taylor_norm(glodap_int_ma_mean,DICfesom_mean,mask=True,title=title)
                
            # fig export  
            if(self.savefig==True):                
                plt.savefig(self.savepath+self.runname+'_'+'DIC_Glodap_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                        dpi = 300, bbox_inches='tight')
            plt.show(block=False)  
                

In [ ]:
### TESTING ###
if __name__ == "__main__":
    
    # run specification -------------------------------------------------------------------------------------
    runname      =  'fesom'
    resultpath = '/work/ollie/mozeisin/results/f2r1.2/' + 'mo20'
    savepath = '/home/ollie/mozeisin/evaluation/mo_files/'

    meshpath = '/work/ollie/mozeisin/mesh/mesh_fesom2.0/core2_meanz'

    # period of analysis ------------------------------------------------------------------------------------
    first_year = 1965
    last_year  = 1966

    # WOA ------------------------------------------------------------------------------------    
    ncfileDIC                = '/work/ollie/projects/MarESys/evaluation/GLODAPv2.2016b.TCO2.nc'
    
    # now test:
    test = DICcomp(runname,resultpath,savepath,meshpath,ncfileDIC,first_year,last_year,
                   layerwise=False,
                   depth_limit = 200,
                   savefig=False)